In [1]:
import json
from datetime import datetime
import re

def format_chat_message(chat_message):
    chat_message = re.sub(r'[\u202F\u2011\u202a\u202c\u200e]', ' ', chat_message)
    # Extract the components of the chat message
    if ']' in chat_message:
        timestamp_str, rest = chat_message.strip().split('] ', 1)
        timestamp_str = timestamp_str[1:]  # Remove the leading '['
        sender, message = rest.split(': ', 1)
    # Check if sender is a phone number or has the prefix "cust"
        if re.match(r'\+\d{1,4}(?:\s\d+)+', sender.strip()) or sender.lower().strip().startswith('cust'):
            sender = 'user'
        else:
            sender = 'agent'    

        # Convert the timestamp to ISO 8601 format
        try:
            # Try 24-hour format first
            timestamp = datetime.strptime(timestamp_str, '%d/%m/%Y, %H:%M:%S').isoformat()
            # print("converting"+timestamp_str+" with 24 hour format")
        except ValueError:
            # If it fails, try 12-hour format with AM/PM
            # print("converting"+timestamp_str+" with 12 hour format")
            timestamp = datetime.strptime(timestamp_str, '%d/%m/%Y, %I:%M:%S %p').isoformat()
        
        # Create the object
        if 'attached' in message and '.jpg' in message or "image omitted" in message:
            return {"timestamp": timestamp,"role": sender.strip(),"message": "sender attached an image"}
        return {"timestamp": timestamp,"role": sender.strip(),"message": message.strip()}
    if 'attached' in chat_message and '.jpg' in chat_message or "image omitted" in chat_message:
        return {"role": None,"message": "sender attached an image"}
    return {"role": None,"message": chat_message.strip()}

# Example usage
# chat_message = "[28/10/2024, 5:01:54 PM] ‪+60 12‑3456 7890‬: ‎Messages and calls are end-to-end encrypted. No one outside of this chat, not even WhatsApp, can read or listen to them."
# formatted_message = format_chat_message(chat_message)
# print(formatted_message)

In [2]:
import json
# read text files in data/chatlogs
files = ['_chat.txt', '_chat 2.txt', '_chat 3.txt']
chatlogs = []
for file in files:
    previous_message = None
    chatlog = []
    with open('../data/chatlogs/'+file, 'r') as f:
        file_messages = f.readlines()
        last_message = file_messages[-1].split(': ')[1].strip()
        for line in file_messages[1:]:
            if "uses a default timer for disappearing messages" in line:
                continue
            if (line.strip().strip('\n') == '' )&(previous_message != None): #concatenate newline only lines
                previous_message["message"] += ' \n '
                continue
            message = format_chat_message(line)
            if previous_message is not None:
                if previous_message['role'] == message['role']:
                    previous_message['message'] += '<|USER_MSG|>' + message['message'] # concatenate consecutive messages
                    if last_message in previous_message['message']:
                        chatlog.append(previous_message)
                elif message['role'] == None:
                    previous_message['message'] += message['message'] #concatenate next line of the same message
                    if last_message in previous_message['message']:
                        chatlog.append(previous_message)
                else:
                    chatlog.append(previous_message)
                    if last_message in message['message']: #detect if last message is detected
                        chatlog.append(message)
                    previous_message = message #switch users
            else: #first message
                previous_message = message
    chatlogs.append(chatlog)
            # if not is_continuation:
            #     print(previous_message)
        

In [26]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_ollama.chat_models import ChatOllama as Ollama
from langchain_core.output_parsers import StrOutputParser as parser

def anonymize_personal_info(data):
    prompt_text = """You are an assistant tasked with translation. \
                    directly translate the given chatlogs to plain english \
                    the chatlogs may contain slangs and shortforms in Malay\
                    Do not include text like "here is an anonymized version" in your responses \
                    Do not change anything else. \
                    Do not change emojis. \
                    Do not provide summaries. \
                    for more context, tlong = tolong, sy or sya = saya, like emojis ~ ok\
                    chatlogs: {element} """
    # prompt_text = """You are an assistant tasked with finding names. \
    #                 if you find any names in the given chatlog messages bracket the names. \
    #                 do not change anything else. \
    #                 Do not include text like "here is an anonymized version" in your responses \
    #                 chatlogs: {element} """

    prompt = ChatPromptTemplate.from_template(prompt_text)
    model = Ollama(temperature=0, model="llama3.2")
    anonymize_chain = {"element": lambda x: x} | prompt | model | parser()
    table_summaries = anonymize_chain.batch([message['message'] for message in data], {"max_concurrency": 5})
    
        
    #text_summaries =  summarize_chain.batch(data_category[0], {"max_concurrency": 5})# no need to summarize

    return table_summaries

In [27]:
for chatlog in chatlogs:
    chatlog = anonymize_personal_info(chatlog)
    for message in chatlog:
        print(message)
        

Hi Maisarah, sorry for the mess earlier, Izzul Khairi from Great Eastern. Do you want to know if you're interested in subscribing to our life insurance, monthly premium is just RM45. We also have a promotion for those who confirm subscribing today, we'll give you an e-wallet of RM50.
Sorry I'm not interested. I've subscribed to insurance. Thank you.
Okeyy, good, thank you very much, I'll respond soon.
👍
I want to ask if you can also promote my school on social media.
My family and I know the insurance company 🤣
Hi YUSNORITA BINTI ABDUL WAHAB @ YUSOF
Hi
Hi, I'm Daz from Great Eastern. Are you interested in getting the RM50 cashback reward? 😊
If you're interested in what I need to do
Hi, if you're interested in signing up for our insurance plan worth RM30. The first month is just RM1. 😊
Oh okay dude
https://greatmultiprotect.com/gss315-spif/

saya: apa kabar bro?
bro: sapa bro? 
saya: saya
bro: apa yang mau saya lakukan hari ini?
saya: saya mau beli laptop baru
bro: kapan kamu mau beli?
